In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from keras.models import load_model
import pandas as pd 
import spacy
#nltk  Stemming
from nltk.stem import PorterStemmer
import time
MAINDIR = "./gdrive/My Drive/EA-test/"
from joblib import load
import sys
sys.path.append(MAINDIR)  #this is only necessary to use libraries not loaded in Google Colab
#to translate tweet into english
from googletrans import Translator
#fake agent to make multiple request and avoid request limit
from fake_useragent import UserAgent
#detect language
from langdetect import detect

Using TensorFlow backend.


In [0]:
language = "en_core_web_sm" #select language for spacy
nlp = spacy.load(language)
ps = PorterStemmer()


In [0]:
#generate fake agent
ua = UserAgent()

In [0]:
#Load file with youtube oomments
df = pd.read_csv(MAINDIR + 'youtube_comments.csv')

In [6]:
df.head()

,Video ID,Title,Comment
0,FVRxaQrj1U4,FIFA 20 Live Reveal – EA PLAY 2019,Why Volta football is not in the switch?????
1,FVRxaQrj1U4,FIFA 20 Live Reveal – EA PLAY 2019,Thes es the best game!!!
2,FVRxaQrj1U4,FIFA 20 Live Reveal – EA PLAY 2019,I got fifa 20
3,FVRxaQrj1U4,FIFA 20 Live Reveal – EA PLAY 2019,Como venden humo.. yo ya lo probe y es una mierda
4,FVRxaQrj1U4,FIFA 20 Live Reveal – EA PLAY 2019,1997 fifa street 2020 fifa 20 volta football


In [0]:
#to detect language in comments
def detect_(txt):
  try:
    return detect(txt)
  except:
    return 'xxxxxxxxxxxxxxxxx'

In [0]:
#create column with language information
temp = df.Comment.apply(detect_)
df['language'] = temp

In [0]:
#discard any comment that has generated error
df2 = df[~(df.language == 'xxxxxxxxxxxxxxxxx')].copy()

In [0]:
#column to store comments in english
df2['text_en'] = None

In [0]:
#get comments that are not in english
df2 = df2[~(df2.language == 'en')]
df2.reset_index(drop=True,inplace=True)

In [0]:
#Function to translate tweet into english
def translate(txt):
  translator = Translator(user_agent=ua.random)
  try:
    trsl = translator.translate(txt)
    return trsl.text
  except:
    print('FAIL')

In [13]:
for i in range(1,687):

  df2.text_en[(i-1)*10:(i)*10] = df2.Comment[(i-1)*10:(i)*10].apply(translate)
  #df2.to_csv('youtube.csv',index=False)
  time.sleep(3)
  break

FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL


In [16]:
df2.text_en[:10] = 'hello world'

0    hello world
1    hello world
2    hello world
3    hello world
4    hello world
5    hello world
6    hello world
7    hello world
8    hello world
9    hello world
Name: text_en, dtype: object

In [0]:
### add comments in english to translated comments
#get english comments
df3 = df[(df.language == 'en')].copy()
df3['text_en'] = df3.Comment

In [0]:
#join english and translated comments
df2 = df3.append(df2[~df2.text_en.isnull()])
df2.reset_index(drop=True, inplace=True)

In [0]:
#df2.to_csv(MAINDIR+'ytComments_translated.csv', index=False) # save result
#df2 = pd.read_csv(MAINDIR+'ytComments_translated.csv') #load result

## Function to preprocess text

In [0]:
#to apply nlp spacy function
def nlp_(txt):
  if type(txt) == str:
    return nlp(txt)
  else:
    return 'xxxxxxxxxxxxxxxxx'

In [0]:
#Preprocess function 
#Exclude words:
wordEX = []
posEx = ['PUNCT','SPACE']
is_stopEx = []
lemmaEx = []
stemEx = []


def text_transf(texts, method = 'lemma_stem'):
  txt2 = []
  for w in texts:
    if (w.text.lower() in wordEX) | (w.pos_ in posEx) | (w.is_stop in is_stopEx) | (w.lemma_ in lemmaEx) | (ps.stem(w.text.lower()) in stemEx):
      continue
    if method == 'word':
      txt2.append(w.text.lower())
    elif method == 'lemma':
      txt2.append(w.lemma_.lower())
    elif method == 'stem':
      txt2.append(ps.stem(w.text.lower()))
    elif method == 'lemma_stem':
      txt2.append(ps.stem(w.lemma_))
      
  return txt2

In [0]:
#join list o result from text_transf in one string
def join_(txt):
  return ' '.join(txt)

## Preprocess text

In [0]:
df2['preprocess'] = df2.text_en.apply(nlp_)

In [0]:
#To delete any row if it fails with the nlp_ function
df2 = df2[~(df2.preprocess == 'xxxxxxxxxxxxxxxxx')]

In [0]:
df2.loc[:,'preprocess'] = df2['preprocess'].apply(text_transf)
df2.loc[:,'preprocess'] = df2['preprocess'].apply(join_)

In [0]:
#load tfidf_model
tfidf_model = load(MAINDIR + 'tfidf.pkl')
#transform text to numeric vector
X_vec = tfidf_model.transform(df2.preprocess)
X_vec = X_vec.toarray()

In [0]:
#Load neural network model
model = load_model(MAINDIR +'NN_model.h5')

In [0]:
#classify vector x_vec
y_pre = model.predict(X_vec)

In [138]:
y_pre

array([[0.91986847, 0.08013149],
       [0.14736857, 0.8526315 ],
       [0.5182465 , 0.48175356],
       ...,
       [0.13382453, 0.8661755 ],
       [0.5182465 , 0.48175356],
       [0.817531  , 0.18246895]], dtype=float32)

In [0]:
#get prediction categories from y_pred
df2['category_id'] = y_pre.argmax(axis=1)
#get positive probability
df2['pos_prob'] = y_pre[:,1]

#Results table

In [144]:
#generate result table
group_location = df2.groupby('Title', as_index=False).agg({'text_en':'count','category_id':'sum'})
group_location.rename(columns={'text_en':'N_comments','category_id':'N_pos'},inplace=True,)
group_location['N_neg'] = group_location.N_comments - group_location.N_pos
group_location['Pos_rate'] = group_location.N_pos/group_location.N_comments
group_location.sort_values('N_comments',ascending=False)

,Title,N_comments,N_pos,N_neg,Pos_rate
8,FIFA 20 Live Reveal – EA PLAY 2019,912,268,644,0.293860
2,DIRECTO DE MODO CARRERA EN FIFA 20 !!!!!,378,47,331,0.124339
9,"FIFA 20 VOLTA STORY MODE, EPISODE 1!! (FIFA 20...",230,84,146,0.365217
10,PRIMER DIRECTO DE FIFA 20 DESDE ALEMANIA !!,165,61,104,0.369697
5,FIFA 20 - Our Training Begins!,106,38,68,0.358491
4,FIFA 20 - Final Preparations,103,39,64,0.378641
3,FIFA 17 OFFICIAL TRAILER,9,3,6,0.333333
0,*LIVE* FIFA 20 DEMO HYPE!!! FIFA 20 DEMO TODAY...,8,5,3,0.625000
1,1ER LIVE FIFA 20 - MODE VOLTA &amp; GAMEPLAY,8,2,6,0.250000
11,REACCIONO al TRAILER de FIFA 20!!,8,3,5,0.375000


We can observe that the percentage of positive comments in most videos is low